In [42]:
import pandas as pd

submission = pd.read_csv("./data/SampleSubmissionStage2.csv")
teams = pd.DataFrame(submission["ID"].str.split("_").to_list())
cols = ["year", "first_team_id", "second_team_id"]
teams.columns = cols
teams = pd.concat([teams, submission], axis=1)
for col in cols:
    teams[col] = teams[col].astype("int64")

saved_teams = teams.copy()

In [43]:
teams = saved_teams.copy()
mteams = pd.read_csv("./data/MTeams.csv")[["TeamID", "TeamName"]]
wteams = pd.read_csv("./data/WTeams.csv")[["TeamID", "TeamName"]]

teams = teams.merge(mteams, left_on="first_team_id", right_on="TeamID", how="left")
teams = teams.merge(wteams, left_on="first_team_id", right_on="TeamID", how="left")


teams["type"] = "M"
teams.loc[teams["TeamName_x"].isna(), "type"] = "W"
teams["first_team_name"] = teams["TeamName_x"].combine_first(teams["TeamName_y"])
teams.drop(columns=["TeamID_x", "TeamName_x", "TeamID_y", "TeamName_y"], inplace=True)

teams = teams.merge(mteams, left_on="second_team_id", right_on="TeamID", how="left")
teams = teams.merge(wteams, left_on="second_team_id", right_on="TeamID", how="left")

teams["second_team_name"] = teams["TeamName_x"].combine_first(teams["TeamName_y"])
teams.drop(columns=["TeamID_x", "TeamName_x", "TeamID_y", "TeamName_y"], inplace=True)

display(teams.isna().sum())
display(teams.type.value_counts())
teams.head()

year                0
first_team_id       0
second_team_id      0
ID                  0
Pred                0
type                0
first_team_name     0
second_team_name    0
dtype: int64

type
M    66066
W    65341
Name: count, dtype: int64

,year,first_team_id,second_team_id,ID,Pred,type,first_team_name,second_team_name
0,2025,1101,1102,2025_1101_1102,0.5,M,Abilene Chr,Air Force
1,2025,1101,1103,2025_1101_1103,0.5,M,Abilene Chr,Akron
2,2025,1101,1104,2025_1101_1104,0.5,M,Abilene Chr,Alabama
3,2025,1101,1105,2025_1101_1105,0.5,M,Abilene Chr,Alabama A&M
4,2025,1101,1106,2025_1101_1106,0.5,M,Abilene Chr,Alabama St


In [46]:
msubmission = teams[teams.type == "M"]
msubmission.to_csv("./interim_data/m_submission.csv", index=False)
wsubmission = teams[teams.type == "W"]
wsubmission.to_csv("./interim_data/w_submission.csv", index=False)
